In [ ]:
import cv2
import numpy as np
import serial

# Initialize serial connection
ser = serial.Serial('COM9', 9600)

# Load your trained model
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

# Get the names of all layers in the network
layer_names = net.getLayerNames()

# Get the indices of the output layers
output_layers_indices = net.getUnconnectedOutLayers()

# Ensure output_layers_indices is a list
if isinstance(output_layers_indices, int):
    output_layers_indices = [output_layers_indices]

# Convert indices to layer names
output_layers = [layer_names[idx - 1] for idx in output_layers_indices]

# Define classes for YOLOv3
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Capture video from webcam
cap = cv2.VideoCapture(0)

while True:
    # Read frame
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame")
        break

    # Resize frame
    frame = cv2.resize(frame, None, fx=0.4, fy=0.4)
    height, width, channels = frame.shape

    # Preprocess the frame
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    # Set input for the network
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Perform Non-Maximum Suppression
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            class_id = class_ids[i]
            label = str(classes[class_id]) if class_id < len(classes) else 'Unknown'
            color = (255, 0, 0)
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label, (x, y + 30), font, 3, color, 2)

            # Send object coordinates to Arduino
            # Adjust the format of the data according to your robotic arm's requirements
            # Here I'm just sending dummy data for illustration
            data_to_send = f"{x},{y},{w},{h}\n"  # Adjust as needed
            ser.write(data_to_send.encode())

    # Show frame
    cv2.imshow("Object Detection", frame)

    # Exit when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
# Close serial connection
ser.close()